# Generate Natural Dataset 2

Follow up from generate natural dataset -> recall output was dataset.csv

In [81]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
import pandas as pd
import os
from data_manipulation_methods import *

In [83]:
data_dir = 'data'

gamma = 0.3
np.random.seed(123) # for deterministic sampling (since generated data is not checked in)

## Step 1: Load Data

In [84]:
full_df = pd.read_csv(os.path.join(data_dir, 'gptGeneratedTexts.csv'))
full_df.head()

,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,cfReview
0,30,3.0,2,True,"08 13, 2016",A2RS58B2MO93F8,1519588135,{'Format:': ' Kindle Edition'},Lisabug1021,I'm not certain how I feel about this book or ...,Good but takes a few strange leaps,1471046400,NaN,I'm uncertain how I feel about this book or th...
1,94,5.0,0,True,"08 23, 2014",A25K7PW9P31CZQ,3979050432,NaN,Daniel Duffield,This is my favorite accessory for when I knock...,Great fit,1408752000,NaN,This is my favorite accessory for when I knock...
2,100,5.0,3,True,"02 22, 2014",A3TKPR7PEI4F9N,3979050432,NaN,Holli Castillo,I'm not sure why my 9th grade daughter wanted ...,"If you like ski masks, this is one for you.",1393027200,NaN,I'm unsure why my 9th grade daughter wanted me...
3,113,5.0,0,True,"03 2, 2016",A8FB0VY4C8AQA,5120053084,"{'Size:': ' X-Large', 'Color:': ' Charcoal'}",Anna @ aGreeneLiving,This will probably be my FAVORITE shirt for br...,This will probably be my FAVORITE shirt for br...,1456876800,NaN,This will probably be my FAVORITE shirt for br...
4,134,3.0,0,True,"02 22, 2018",AL89KEFP125NA,5120053084,"{'Size:': ' Large', 'Color:': ' Berry'}",Sara Murphy,I wasn't such a fan of the unforgiving fabric ...,Not a fan of the fabric,1519257600,NaN,NaN


## Step 2: Only keep those with successful conversion

In [85]:
filtered_df = full_df[~(full_df['cfReview'].isna())]
print(f"Remaining data count: {filtered_df.shape[0]}")
filtered_df.head()

Remaining data count: 1171


,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,cfReview
0,30,3.0,2,True,"08 13, 2016",A2RS58B2MO93F8,1519588135,{'Format:': ' Kindle Edition'},Lisabug1021,I'm not certain how I feel about this book or ...,Good but takes a few strange leaps,1471046400,NaN,I'm uncertain how I feel about this book or th...
1,94,5.0,0,True,"08 23, 2014",A25K7PW9P31CZQ,3979050432,NaN,Daniel Duffield,This is my favorite accessory for when I knock...,Great fit,1408752000,NaN,This is my favorite accessory for when I knock...
2,100,5.0,3,True,"02 22, 2014",A3TKPR7PEI4F9N,3979050432,NaN,Holli Castillo,I'm not sure why my 9th grade daughter wanted ...,"If you like ski masks, this is one for you.",1393027200,NaN,I'm unsure why my 9th grade daughter wanted me...
3,113,5.0,0,True,"03 2, 2016",A8FB0VY4C8AQA,5120053084,"{'Size:': ' X-Large', 'Color:': ' Charcoal'}",Anna @ aGreeneLiving,This will probably be my FAVORITE shirt for br...,This will probably be my FAVORITE shirt for br...,1456876800,NaN,This will probably be my FAVORITE shirt for br...
5,138,5.0,14,True,"03 30, 2016",A2RV03NRVNBLQF,5120053645,"{'Size:': ' 5 - X-Large', 'Color:': ' Grey'}",Kindle Customer,Quite surprised and very happy . I say this b...,Quite surprised and very happy .,1459296000,NaN,"This is well made, expensive and sturdy rayon...."


In [86]:
# clean data

filtered_df['vote'] = filtered_df['vote'].astype(str).str.replace(',','', regex=True).astype(float)
filtered_df['vote'].fillna(0, inplace=True)
filtered_df['vote'] = filtered_df['vote'].astype(int)

# check if all converted to digits
N = filtered_df.shape[0]
is_digit_count = filtered_df['vote'].astype(str).str.isdigit().sum()
print(f'Full size ={N}, Is digit size = {is_digit_count}')

Full size =1171, Is digit size = 1171


/tmp/ipykernel_23343/3233072925.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['vote'] = filtered_df['vote'].astype(str).str.replace(',','', regex=True).astype(float)
/tmp/ipykernel_23343/3233072925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['vote'].fillna(0, inplace=True)
/tmp/ipykernel_23343/3233072925.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

## Step 3: Data balancing

Want: $P(Y=1|Z=1) = P(Y=0|Z=0) = \gamma$

See Appendix -> B.2.1

In [87]:
# Extra step, randomly perterb the data since we only got with not cases

N = filtered_df.shape[0]

# flip half
simObsText = filtered_df['reviewText'].to_numpy()
simOcFText = filtered_df['cfReview'].to_numpy()
simObsFlag = np.ones(N, dtype=bool)

print(int(N/2))
to_flip = np.random.random(N) > 0.5
simObsText[to_flip] = filtered_df.iloc[to_flip]['cfReview'].to_numpy()
simOcFText[to_flip] = filtered_df.iloc[to_flip]['reviewText'].to_numpy()
simObsFlag[to_flip] = ~simObsFlag[to_flip]

filtered_df = filtered_df.assign(simObserved = simObsText)
filtered_df = filtered_df.assign(simCounterFact = simOcFText)
filtered_df = filtered_df.assign(extra_not = simObsFlag)

not_flipped = filtered_df['extra_not']
print(f"Not Flipped={not_flipped.sum()}, Flipped={(~not_flipped).sum()}")
filtered_df.head()

585
Not Flipped=605, Flipped=566


,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,cfReview,simObserved,simCounterFact,extra_not
0,30,3.0,2,True,"08 13, 2016",A2RS58B2MO93F8,1519588135,{'Format:': ' Kindle Edition'},Lisabug1021,I'm uncertain how I feel about this book or th...,Good but takes a few strange leaps,1471046400,NaN,I'm uncertain how I feel about this book or th...,I'm uncertain how I feel about this book or th...,I'm uncertain how I feel about this book or th...,False
1,94,5.0,0,True,"08 23, 2014",A25K7PW9P31CZQ,3979050432,NaN,Daniel Duffield,This is my favorite accessory for when I knock...,Great fit,1408752000,NaN,This is my favorite accessory for when I knock...,This is my favorite accessory for when I knock...,This is my favorite accessory for when I knock...,True
2,100,5.0,3,True,"02 22, 2014",A3TKPR7PEI4F9N,3979050432,NaN,Holli Castillo,I'm not sure why my 9th grade daughter wanted ...,"If you like ski masks, this is one for you.",1393027200,NaN,I'm unsure why my 9th grade daughter wanted me...,I'm not sure why my 9th grade daughter wanted ...,I'm unsure why my 9th grade daughter wanted me...,True
3,113,5.0,0,True,"03 2, 2016",A8FB0VY4C8AQA,5120053084,"{'Size:': ' X-Large', 'Color:': ' Charcoal'}",Anna @ aGreeneLiving,This will probably be my FAVORITE shirt for br...,This will probably be my FAVORITE shirt for br...,1456876800,NaN,This will probably be my FAVORITE shirt for br...,This will probably be my FAVORITE shirt for br...,This will probably be my FAVORITE shirt for br...,False
5,138,5.0,14,True,"03 30, 2016",A2RV03NRVNBLQF,5120053645,"{'Size:': ' 5 - X-Large', 'Color:': ' Grey'}",Kindle Customer,"This is well made, expensive and sturdy rayon....",Quite surprised and very happy .,1459296000,NaN,"This is well made, expensive and sturdy rayon....","This is well made, expensive and sturdy rayon....","This is well made, expensive and sturdy rayon....",False


In [88]:
# 1. drop reviews with 0 helpful votes (V) until
# P(V>0|Z=0) > 1 - gamma

print('Before:')

num_haveNots = (filtered_df['extra_not']).sum()
num_haveNot_hasVote = (filtered_df[filtered_df['extra_not']]['vote'] > 0).sum()
print(f'num Z=1 = {num_haveNots}')
print(f'num V>0|Z=1 = {num_haveNot_hasVote}')
print(f'P(V>0|Z=1) = {num_haveNot_hasVote/num_haveNots}')
print(f'gamma = {gamma}')

num_minNots = (~filtered_df['extra_not']).sum()
num_minNots_hasVote = (filtered_df[~filtered_df['extra_not']]['vote'] > 0).sum()
print(f'num Z=0 = {num_minNots}')
print(f'num V>0|Z=0 = {num_minNots_hasVote}')
print(f'P(V>0|Z=0) = {num_minNots_hasVote/num_minNots}')
print(f'1 - gamma = {1 - gamma}')


drop_data(filtered_df, gamma=gamma,
          Z_label='extra_not', V_label='vote')


print('\n\nAfter:')

num_haveNots = (filtered_df['extra_not']).sum()
num_haveNot_hasVote = (filtered_df[filtered_df['extra_not']]['vote'] > 0).sum()
print(f'num Z=1 = {num_haveNots}')
print(f'num V>0|Z=1 = {num_haveNot_hasVote}')
print(f'P(V>0|Z=1) = {num_haveNot_hasVote/num_haveNots}')
print(f'gamma = {gamma}')

num_minNots = (~filtered_df['extra_not']).sum()
num_minNots_hasVote = (filtered_df[~filtered_df['extra_not']]['vote'] > 0).sum()
print(f'num Z=0 = {num_minNots}')
print(f'num V>0|Z=0 = {num_minNots_hasVote}')
print(f'P(V>0|Z=0) = {num_minNots_hasVote/num_minNots}')
print(f'1 - gamma = {1 - gamma}')

Before:
num Z=1 = 605
num V>0|Z=1 = 92
P(V>0|Z=1) = 0.15206611570247933
gamma = 0.3
num Z=0 = 566
num V>0|Z=0 = 81
P(V>0|Z=0) = 0.1431095406360424
1 - gamma = 0.7


After:
num Z=1 = 123
num V>0|Z=1 = 92
P(V>0|Z=1) = 0.7479674796747967
gamma = 0.3
num Z=0 = 115
num V>0|Z=0 = 81
P(V>0|Z=0) = 0.7043478260869566
1 - gamma = 0.7


In [89]:
# 2. find smallest Tz s.t. P(V>T1|Z=1) < gamma and P(V>T0|Z=0) < 1-gamma

T_1, T_0 = find_smallest_Tz(filtered_df, gamma=gamma, Z_label='extra_not', V_label='vote')

num_haveNots = (filtered_df['extra_not']).sum()
num_haveNots_abvT1 = (filtered_df[filtered_df['extra_not']]['vote'] > T_1).sum()
print(f'T_1 = {T_1}')
print(f'num Z=1 = {num_haveNots}')
print(f'num V>T1|Z=1 = {num_haveNots_abvT1}')
print(f'P(V>T1|Z=1) = {num_haveNots_abvT1/num_haveNots}')
print(f'gamma = {gamma}')

print('\n')

num_minNots = (~filtered_df['extra_not']).sum()
num_minNots_abvT0 = (filtered_df[~filtered_df['extra_not']]['vote'] > T_0).sum()
print(f'T_0 = {T_0}')
print(f'num Z=0 = {num_minNots}')
print(f'num V>T0|Z=0 = {num_minNots_abvT0}')
print(f'P(V>T0|Z=0) = {num_minNots_abvT0/num_minNots}')
print(f'1 - gamma = {1 - gamma}')

T_1 = 4
num Z=1 = 123
num V>T1|Z=1 = 35
P(V>T1|Z=1) = 0.2845528455284553
gamma = 0.3


T_0 = 2
num Z=0 = 115
num V>T0|Z=0 = 63
P(V>T0|Z=0) = 0.5478260869565217
1 - gamma = 0.7


In [90]:
# 3. set Y=1[V>T0] for each Z=0, Y=1[>T1] for each Z=1
# 4. random flip Y=0 to Y=1 in examples where (Z=0,V=T0+1) or (Z=1,V=T1+1), until
# P(Y=1|Z=1) > gamma and P(Y=1|Z=0) > 1-gamma

filtered_df = assign_natural_Y(filtered_df, T_1=T_1, T_0=T_0,
                               gamma=gamma,
                               Z_label='extra_not', V_label='vote')

num_haveNots = (filtered_df['extra_not']).sum()
num_above3stars_abvTH = (filtered_df[filtered_df['extra_not']]['aboveVThreshold']).sum()
print(f'num Z=1 = {num_haveNots}')
print(f'num Y=1|Z=1 = {num_above3stars_abvTH}')
print(f'P(Y=1|Z=1) = {num_above3stars_abvTH/num_haveNots}')
print(f'gamma = {gamma}')

print('\n')

num_minNots = (~filtered_df['extra_not']).sum()
num_leq3stars_abvTH = (filtered_df[~filtered_df['extra_not']]['aboveVThreshold']).sum()
print(f'num Z=0 = {num_minNots}')
print(f'num Y=1|Z=0 = {num_leq3stars_abvTH}')
print(f'P(Y=1|Z=0) = {num_leq3stars_abvTH/num_minNots}')
print(f'1 - gamma = {1 - gamma}')

num Z=1 = 123
num Y=1|Z=1 = 37
P(Y=1|Z=1) = 0.3008130081300813
gamma = 0.3


num Z=0 = 115
num Y=1|Z=0 = 81
P(Y=1|Z=0) = 0.7043478260869566
1 - gamma = 0.7


## Step 4: Split data to train, val and test set

In [91]:
train_df, test_df = split_dataset(filtered_df, gamma=gamma,
                                  Y_label='aboveVThreshold', Z_label='extra_not')

train_df, val_df = split_dataset(train_df, gamma=gamma,
                                 Y_label='aboveVThreshold', Z_label='extra_not')

print("Train size:", train_df.shape[0], "Val size:", val_df.shape[0], "Test size:", test_df.shape[0])
N_Z1 = train_df['extra_not'].sum()
N_Y1_Z1 = (train_df['extra_not'] & (train_df['aboveVThreshold'])).sum()
P_Y1_Z1 = N_Y1_Z1 / N_Z1
N_Z0 = (~train_df['extra_not']).sum()
N_Y0_Z0 = ((~train_df['extra_not']) & (train_df['aboveVThreshold'])).sum()
P_Y0_Z0 = N_Y0_Z0 / N_Z0
print("N_Z1:", N_Z1,"N_Z0:",N_Z0)
print("N_Z1_Y1", N_Y1_Z1, "N_Z0_Y0",N_Y0_Z0)
print("P(Y1|Z1):",P_Y1_Z1,"P(Y0|Z0):", P_Y0_Z0)

Train size: 152 Val size: 38 Test size: 48
N_Z1: 79 N_Z0: 73
N_Z1_Y1 24 N_Z0_Y0 51
P(Y1|Z1): 0.3037974683544304 P(Y0|Z0): 0.6986301369863014


## Step 5: Save data to .npz format

In [92]:
train_df.head()

,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,cfReview,simObserved,simCounterFact,extra_not,aboveVThreshold
771,54155,5.0,0,True,"06 3, 2016",A1VEQ7NJU3MOCK,B0001YRWJ2,"{'Size:': ' 30W x 30L', 'Color:': ' Indigo Rig...",Roger Kuo,Standard. Do not buy bigger than your own size...,Five Stars,1464912000,NaN,Haven't received this shipment yet. Have bough...,Standard. Do not buy bigger than your own size...,Haven't received this shipment yet. Have bough...,True,False
225,11706,5.0,5,True,"08 13, 2008",A36S0UIHXYL8GD,B0000864HS,NaN,J. Gale,"I've worn this style bra most of my life, goin...",Half Slips Are Hard to Find!,1218585600,NaN,"I've worn this style bra most of my life, goin...","I've worn this style bra most of my life, goin...","I've worn this style bra most of my life, goin...",False,True
1105,81522,3.0,2,True,"05 29, 2012",A2LA59CEMV78BC,B0002LINM4,"{'Size:': ' 7', 'Color:': ' Leopard'}",Jennifer H.,"Apparently, Jockey makes different styles of u...","Ok, but not what I expected",1338249600,NaN,Feels good walking around the office although ...,"Apparently, Jockey makes different styles of u...",Feels good walking around the office although ...,True,False
590,38829,5.0,3,True,"01 31, 2016",A1I24XH740MED9,B0000WL0XY,"{'Size:': ' Large', 'Color:': ' Brown'}",Bluto,"This is a GREAT SHOE, with GREAT foot support,...","Great jacket, get the one size smaller than yo...",1454198400,NaN,"This is a GREAT SHOE, with GREAT foot support,...","This is a GREAT SHOE, with GREAT foot support,...","This is a GREAT SHOE, with GREAT foot support,...",False,True
366,16739,4.0,0,True,"03 21, 2016",A35BJGXH34CDYJ,B00008IQ74,"{'Size:': ' C', 'Color:': ' Barely There', 'Nu...",La Mancha,If I could find something better than this bra...,"I always buy these, but wish I could find better.",1458518400,NaN,I've bought half a dozen of these K87 shirts o...,If I could find something better than this bra...,I've bought half a dozen of these K87 shirts o...,True,False


In [95]:
out_dir = './data'
header_name = 'header'
# train_ds_name = 'notty_train' # full dataset name
# val_ds_name = 'notty_val' # full dataset name
# test_ds_name = 'notty_test'
train_ds_name = 'notty_train_large' # large dataset name
val_ds_name = 'notty_val_large'
test_ds_name = 'notty_test_large'
# train_ds_name = 'notty_train_small' # small dataset name
# val_ds_name = 'notty_val_small' # large dataset name
# test_ds_name = 'notty_test_small'

save_df(train_df, os.path.join(out_dir, train_ds_name), text_keys=['simObserved', 'simCounterFact'])
save_df(val_df, os.path.join(out_dir, val_ds_name), text_keys=['simObserved', 'simCounterFact'])
save_df(test_df, os.path.join(out_dir, test_ds_name), text_keys=['simObserved', 'simCounterFact'])